### <u>Cross-Clustered Models</u>
The varying effects with province based clustering were pretty decent, but some of the data like e.g. int the CARB or NWCS provinces show some complex structure that might benefit from more complex grouping. Moreover, both model comparison methods, the WAIC and LOOCV, flagged max(blue) based clustering as potentially useful in combination with the province-based clustering. In this post I will further the analysis with two cross-classified models, meaning that two types of groupings are applied in parallel, rather than hierarchically. The first model combines the max(blue) grouping with the province grouping, with covarying effects implemented for both groupings, since the covarying effects model proved to be the superior one. For good measure, I'll implement a second model combining max(blue) and biome grouping.

In [ ]:
import pickle
# Importing libraries
import pymc3 as pm
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
from seaborn import PairGrid, kdeplot, jointplot, FacetGrid
import theano.tensor as tt
import re
from matplotlib import cm
from cmocean import cm as cmo
from matplotlib.colors import rgb2hex

In [ ]:
# Setting graphics...
% matplotlib inline
pl.style.use('bmh')
rcParams['xtick.labelsize'] = 16
rcParams['ytick.labelsize'] = 16
rcParams['axes.formatter.limits'] = (-2, 3)
rcParams['axes.labelsize'] = 16
rcParams['font.size'] = 16
rcParams['figure.titlesize'] = 20
rcParams['axes.titlesize'] = 18